In [2]:
# https://www.jeffreythompson.org/collision-detection/table_of_contents.php

In [3]:
# https://www.jeffreythompson.org/collision-detection/line-line.php

In [4]:

x1 = 0    # line controlled by mouse
y1 = 0
x2 = 10   # fixed end
y2 = 10

x3 = 100;  # static line
y3 = 300;
x4 = 500;
y4 = 100;




uA = ((x4-x3)*(y1-y3) - (y4-y3)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));

uB = ((x2-x1)*(y1-y3) - (y2-y1)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));





In [5]:
uA,uB

(23.333333333333332, 0.3333333333333333)

In [29]:
import numpy as np
import keras
from keras.layers import Lambda
import tensorflow as tf


In [221]:
fov = 30* ((2*np.pi)/360.0)
ray_count = 4

p_pos = np.array([10,50])
p_dir = np.array([1.,0.])

walls = tf.stack([[[70,0,70,100],
                   [80,0,50,100]]]) 
walls_count = len(walls[0])




# ray generator
input_p = keras.layers.Input(shape=(2,2),batch_size=1)

n_pos = input_p[0][0]
n_dir = input_p[0][1]


allvect = []


for ray_id in range(-ray_count//2,ray_count//2):

    angle = ((ray_id/ray_count))*fov 
    rotation_matrix = tf.stack([(tf.cos(angle), -tf.sin(angle)), (tf.sin(angle), tf.cos(angle))], axis=0)
       
    n_dir_rot = tf.matmul(tf.reshape(n_dir,(1,2)), rotation_matrix)
    n_dir_rot = tf.reshape(n_dir_rot,(2,) ) *100
    
    end_dir = n_pos + n_dir_rot
    allvect.append( [n_pos,end_dir])
    

first_half = keras.backend.stack(allvect)
#first_half = keras.layers.Concatenate([n_pos,end_dir,])
#first_half = keras.layers.Reshape((ray_count,4,))(first_half)

ray_gen = keras.Model(inputs=input_p, outputs= tf.reshape(first_half,(1,ray_count,4)))



In [222]:



rays = ray_gen.predict(np.array([[p_pos,p_dir]]))

In [223]:
rays

array([[[ 10.      ,  50.      , 106.59258 ,  75.881905],
        [ 10.      ,  50.      , 109.144485,  63.05262 ],
        [ 10.      ,  50.      , 110.      ,  50.      ],
        [ 10.      ,  50.      , 109.144485,  36.94738 ]]], dtype=float32)

In [231]:
input_rays = keras.layers.Input(shape=(ray_count,4),batch_size=1)
input_walls = keras.layers.Input(shape=(walls_count,4),batch_size=1)

#mid = keras.layers.Activation(keras.activations.backend.cos)(input_p)
#mid = input_p[0][0]-input_p[0][1]

outputs=[]
outidx = []
for ridx,ray in enumerate(input_rays[0]):
    
    x1 = ray[0]
    y1 = ray[1]
    x2 = ray[2]
    y2 = ray[3]
    
    for widx,wall in enumerate(input_walls[0]):
        

        x3 = wall[0]
        y3 = wall[1]
        x4 = wall[2]
        y4 = wall[3]

        uA = ((x4-x3)*(y1-y3) - (y4-y3)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));
        uB = ((x2-x1)*(y1-y3) - (y2-y1)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));
        
        
        outputs.append([uA,uB])
        
        outidx.append([ridx,widx])

    

outputs = keras.backend.stack(outputs)
outidx = keras.backend.stack(outidx)
outputs = tf.reshape(outputs,(ray_count,walls_count,2))
#
m = keras.Model(inputs=(input_rays,input_walls), outputs=outputs)

In [232]:
vecs.shape

(1, 12, 4)

In [239]:

rays = ray_gen.predict(np.array([[p_pos,p_dir]]))

# ray_count,walls_count,2
colis = m.predict((rays,walls))


<tf.Tensor: shape=(4, 2, 2), dtype=float32, numpy=
array([[[0.62116575, 0.6607695 ],
        [0.52703625, 0.636407  ]],

       [[0.6051774 , 0.57899153],
        [0.5336683 , 0.5696577 ]],

       [[0.6       , 0.5       ],
        [0.55      , 0.5       ]],

       [[0.6051774 , 0.4210085 ],
        [0.577557  , 0.4246137 ]]], dtype=float32)>

In [258]:
colis[1]

array([[0.6051774 , 0.57899153],
       [0.5336683 , 0.5696577 ]], dtype=float32)

In [259]:
np.argmin(colis[1],axis=0)

array([1, 1], dtype=int64)

In [262]:
tf.argmin(colis,axis=1)

<tf.Tensor: shape=(4, 2), dtype=int64, numpy=
array([[1, 1],
       [1, 1],
       [1, 0],
       [1, 0]], dtype=int64)>

In [238]:
colis

array([[[0.62116575, 0.6607695 ],
        [0.52703625, 0.636407  ]],

       [[0.6051774 , 0.57899153],
        [0.5336683 , 0.5696577 ]],

       [[0.6       , 0.5       ],
        [0.55      , 0.5       ]],

       [[0.6051774 , 0.4210085 ],
        [0.577557  , 0.4246137 ]]], dtype=float32)